In [25]:
import cv2
import tensorflow as tf
import numpy as np
import os
from mtcnn.mtcnn import MTCNN

# Crop photo with detector harcascade or mtcnn

In [26]:
detector_harcascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [27]:
def detect_face(img):
    pixels = np.asarray(img)
    results = detector_harcascade.detectMultiScale(img,1.2,4)
    if len(results) > 0:
        x1,y1,w,h = results[0]
        
        x1, y1 = abs(x1),abs(y1)
        x2,y2 = x1 + w, y1 + h
        img = pixels[y1:y2,x1:x2]
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img,(224,224))
        return img



def print_prog(val, val_len, folder, bar_size=20):
    progr = "="*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end="\n")
    else:
        print("[%s] (%d samples) \t label : %s\t\t" % (progr, val+1,folder), end="\r")

def img_augmentasi(img):
    h,w = img.shape[:2]
    center = (w//2,h//2)
    
    M_rot5 = cv2.getRotationMatrix2D(center,5,1.0)
    M_rot_neg_5 = cv2.getRotationMatrix2D(center,-5,1.0)
    M_rot_10 = cv2.getRotationMatrix2D(center,10,1.0)
    M_rot_neg_10 = cv2.getRotationMatrix2D(center,-10,1.0)
    M_trans_3=np.float32([[1,0,3], [0,1,0]])
    M_trans_neg_3=np.float32([[1,0,-3], [0,1,0]])
    M_trans_6=np.float32([[1,0,6], [0,1,0]])
    M_trans_neg_6=np.float32([[1,0,-6], [0,1,0]])
    M_trans_y3=np.float32([[1,0,0], [0,1,3]])
    M_trans_neg_y3=np.float32([[1,0,0], [0,1,-3]])
    M_trans_y6=np.float32([[1,0,0], [0,1,6]])
    M_trans_neg_y6=np.float32([[1,0,0], [0,1,-6]])
    
    imgs=[]
    imgs.append(cv2.warpAffine(img, M_rot5,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_rot_neg_5,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_rot_10,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_rot_neg_10,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_trans_3,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_trans_neg_3,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_trans_6,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_trans_neg_6,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_trans_y3,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_trans_neg_y3,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_trans_y6,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.warpAffine(img, M_trans_neg_y6,(w,h),borderValue=(255,255,255)))
    imgs.append(cv2.add(img, 10))
    imgs.append(cv2.add(img, 30))
    imgs.append(cv2.add(img, -10))
    imgs.append(cv2.add(img, -30))
    imgs.append(cv2.add(img, 15))
    imgs.append(cv2.add(img, 45))
    imgs.append(cv2.add(img, -15))
    imgs.append(cv2.add(img, -45))
    
    
    return imgs
    

In [28]:
data_sets = "Datasets10/train_new/"

names = []
images = []


for folder in os.listdir(data_sets):
    files = os.listdir(os.path.join(data_sets,folder))[:50]
    for i, name in enumerate(files):
        if (name.find(".jpg") > -1) or (name.find(".JPG") > -1):
            img = cv2.imread(os.path.join(data_sets + folder ,name))
            img = detect_face(img)
            if img is not None :
                images.append(img)
                names.append(folder)
                
                print_prog(i,len(files), folder)

[====================] (50 samples) 	 label : .Uknown		
[=================   ] (43 samples) 	 label : Azizi N		
[====================] (50 samples) 	 label : Bhara M		
[====================] (50 samples) 	 label : Cristiano Ronaldo		


In [29]:
augmentasi_img = []
augmentasi_names = []
for i, img in enumerate(images):
    try:
        augmentasi_img.extend(img_augmentasi(img))
        augmentasi_names.extend([names[i]] * 20)
    except:
        print(i)

In [30]:
images.extend(augmentasi_img)
names.extend(augmentasi_names)

In [31]:
len(augmentasi_img), len(augmentasi_names)

(4580, 4580)

In [32]:
len(names), len(images)

(4809, 4809)

In [33]:
unique , counts = np.unique(names,return_counts= True)
for item in zip(unique,counts):
    
    print(item)

('.Uknown', 1029)
('Azizi N', 756)
('Bhara M', 693)
('Cristiano Ronaldo', 630)
('Lionel Messi', 672)
('Tony_Blair', 1029)


In [34]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
le.fit(names)
labels = le.classes_
name_vec = le.transform(names)

categorical_name_vec = tf.keras.utils.to_categorical(name_vec)

In [35]:
print("number of class :", len(labels))
print(labels)

number of class : 6
['.Uknown' 'Azizi N' 'Bhara M' 'Cristiano Ronaldo' 'Lionel Messi'
 'Tony_Blair']


In [36]:
x_train,x_test,y_train,y_test = train_test_split(np.array(images, dtype=np.float32),
                                                np.array(categorical_name_vec),
                                                test_size=0.15,random_state=42)
print(x_train[0].shape, y_train.shape, x_test.shape,y_test.shape)

(224, 224) (4087, 6) (722, 224, 224) (722, 6)


In [37]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)

x_train.shape, x_test.shape, x_train[0].shape

((4087, 224, 224, 1), (722, 224, 224, 1), (224, 224, 1))

In [38]:
model = tf.keras.models.Sequential(
    [
     tf.keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(224,224,1)),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Dropout(0.2),
        
     tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Dropout(0.3),
     
        
     tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Dropout(0.4),
     
        
     tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Dropout(0.4),
        
     tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Dropout(0.5),
     
     tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Dropout(0.5),    

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dropout(0.5),
     tf.keras.layers.Dense(1024, activation='relu'),

     tf.keras.layers.Dense(len(labels),activation='softmax')                                   
    ]
)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 222, 222, 16)      160       
_________________________________________________________________
batch_normalization_30 (Batc (None, 222, 222, 16)      64        
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 109, 109, 32)      4640      
_________________________________________________________________
batch_normalization_31 (Batc (None, 109, 109, 32)      128       
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 54, 54, 32)       

In [39]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [40]:
BATCH_SIZE = 32
EPOCH = 50
STEPS_PER_EPOCH = 16
history = model.fit(x_train,
                    y_train,
                    epochs=EPOCH,
                    batch_size= BATCH_SIZE,
                    validation_split=0.2,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    verbose=1
                   )

Epoch 1/50
16/16 [==============================] - 94s 6s/step - loss: 1.7514 - accuracy: 0.3331 - val_loss: 2.9572 - val_accuracy: 0.1589
Epoch 2/50
16/16 [==============================] - 97s 6s/step - loss: 1.3262 - accuracy: 0.4818 - val_loss: 1.7050 - val_accuracy: 0.2286
Epoch 3/50
16/16 [==============================] - 100s 6s/step - loss: 1.1168 - accuracy: 0.5662 - val_loss: 1.5678 - val_accuracy: 0.2702
Epoch 4/50
16/16 [==============================] - 96s 6s/step - loss: 0.9210 - accuracy: 0.6222 - val_loss: 1.8189 - val_accuracy: 0.3227
Epoch 5/50
16/16 [==============================] - 100s 6s/step - loss: 0.8044 - accuracy: 0.6568 - val_loss: 2.0355 - val_accuracy: 0.3594
Epoch 6/50
16/16 [==============================] - 98s 6s/step - loss: 0.7058 - accuracy: 0.6895 - val_loss: 1.6637 - val_accuracy: 0.3692
Epoch 7/50
16/16 [==============================] - 95s 6s/step - loss: 0.6360 - accuracy: 0.7213 - val_loss: 2.7434 - val_accuracy: 0.3411
Epoch 8/50
16/16 [

In [41]:
model.save('model_batch_face.h5')

In [41]:
model.layers